In [1]:
%%time
%%sh

# The name of our algorithm
# Makoto.Sano@Mack-the-Psych.com
algorithm_name=sagemaker-vdok3-bert-cv-def

cd container

chmod +x vdok3_sage/train
chmod +x vdok3_sage/serve

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build  -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

Login Succeeded
Sending build context to Docker daemon  40.45kB
Step 1/32 : FROM nvidia/cuda:10.0-cudnn7-devel-ubuntu16.04
10.0-cudnn7-devel-ubuntu16.04: Pulling from nvidia/cuda
528184910841: Pulling fs layer
8a9df81d603d: Pulling fs layer
636d9303bf66: Pulling fs layer
672b5bdcef61: Pulling fs layer
9061ebd9855f: Pulling fs layer
4829f3972b02: Pulling fs layer
c473884057c7: Pulling fs layer
8383fa66049c: Pulling fs layer
02a3bdb75881: Pulling fs layer
95c482736993: Pulling fs layer
4834d61defc0: Pulling fs layer
72bd608d1baa: Pulling fs layer
6b5a3a075e9d: Pulling fs layer
8383fa66049c: Waiting
672b5bdcef61: Waiting
9061ebd9855f: Waiting
4829f3972b02: Waiting
02a3bdb75881: Waiting
95c482736993: Waiting
4834d61defc0: Waiting
72bd608d1baa: Waiting
c473884057c7: Waiting
6b5a3a075e9d: Waiting
8a9df81d603d: Verifying Checksum
8a9df81d603d: Download complete
636d9303bf66: Verifying Checksum
636d9303bf66: Download complete
672b5bdcef61: Download complete
9061ebd9855f: Verifying Checksum
906

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



CPU times: user 137 ms, sys: 32.2 ms, total: 170 ms
Wall time: 22min 36s


In [2]:
# S3 prefix
# Makoto.Sano@Mack-the-Psych.com
prefix = 'vdok3_bert_cv_def'

# Define IAM role
import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()

In [3]:
import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()

In [4]:
WORK_DIRECTORY = 'data'

data_location = sess.upload_data(WORK_DIRECTORY, key_prefix=prefix)

In [5]:
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name

# Makoto.Sano@Mack-the-Psych.com
image = '{}.dkr.ecr.{}.amazonaws.com/sagemaker-vdok3-bert-cv-def:latest'.format(account, region)
vdok3bert = sage.estimator.Estimator(image,
                       role, 1, 'ml.p3.2xlarge',
                       output_path="s3://{}/output".format(sess.default_bucket()),
                       sagemaker_session=sess)
vdok3bert.fit(data_location)

2021-08-17 01:35:50 Starting - Starting the training job...
2021-08-17 01:35:54 Starting - Launching requested ML instancesProfilerReport-1629164150: InProgress
......
2021-08-17 01:37:11 Starting - Preparing the instances for training......
2021-08-17 01:38:19 Downloading - Downloading input data...
2021-08-17 01:38:51 Training - Downloading the training image......................../.pyenv/versions/anaconda3-5.0.0/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Starting the training.
bert.embeddings.word_embeddings.weight→embeddings.word_embeddings.weight
bert.embeddings.position_embeddings.weight→embeddings.position_embeddings.weight
bert.embeddings.token_type_embeddings.weight→embeddings.token_type_embeddings.weight
bert.embeddings.LayerNorm

In [6]:
from sagemaker.predictor import csv_serializer
predictor = vdok3bert.deploy(1, 'ml.p3.2xlarge', serializer=csv_serializer)

-------------------!

In [7]:
df_in = pd.read_csv('data/Head4-Serialized-Def-ELVA.PILOT.POST-TEST.csv')
np_in = np.vstack((np.array(df_in.columns), df_in.to_numpy()))
print(predictor.predict(np_in).decode('utf-8'))

The csv_serializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


Score_Class,Definition-Score
0,0
1,1
1,1
2,2



In [8]:
sess.delete_endpoint(predictor.endpoint)

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [9]:
transform_output_folder = "batch-transform-output"
output_path="s3://{}/{}".format(sess.default_bucket(), transform_output_folder)

transformer = vdok3bert.transformer(instance_count=1,
                               instance_type='ml.p3.2xlarge',
                               output_path=output_path,
                               assemble_with='Line',
                               accept='text/csv')

In [10]:
transformer.transform(data_location + '/Head4-Serialized-Def-ELVA.PILOT.POST-TEST.csv', content_type='text/csv', split_type='Line')
transformer.wait()

s3_client = sess.boto_session.client('s3')
s3_client.download_file(sess.default_bucket(), "{}/Head4-Serialized-Def-ELVA.PILOT.POST-TEST.csv.out".format(transform_output_folder), '/tmp/Head4-Serialized-Def-ELVA.PILOT.POST-TEST.csv.out')
with open('/tmp/Head4-Serialized-Def-ELVA.PILOT.POST-TEST.csv.out') as f:
    results = f.readlines()   
print("Transform results: \n{}".format(''.join(results)))

..............................................Starting the inference server with 8 workers.
[2021-08-17 02:14:17 +0000] [41] [INFO] Starting gunicorn 20.1.0
[2021-08-17 02:14:17 +0000] [41] [INFO] Listening at: unix:/tmp/gunicorn.sock (41)
[2021-08-17 02:14:17 +0000] [41] [INFO] Using worker: gevent
[2021-08-17 02:14:17 +0000] [45] [INFO] Booting worker with pid: 45
[2021-08-17 02:14:17 +0000] [46] [INFO] Booting worker with pid: 46
[2021-08-17 02:14:17 +0000] [54] [INFO] Booting worker with pid: 54
[2021-08-17 02:14:17 +0000] [55] [INFO] Booting worker with pid: 55
[2021-08-17 02:14:17 +0000] [63] [INFO] Booting worker with pid: 63
[2021-08-17 02:14:17 +0000] [72] [INFO] Booting worker with pid: 72
[2021-08-17 02:14:18 +0000] [86] [INFO] Booting worker with pid: 86
[2021-08-17 02:14:18 +0000] [89] [INFO] Booting worker with pid: 89
/.pyenv/versions/anaconda3-5.0.0/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `floa